# Capstone Project - Identify new location for Indian Restaurant in Kuala Lumpur, Malaysia.

## Table of contents
* [Introduction: Business Problem Section](#introduction)
* [Data Collection](#data)
* [Analysis](#analysis)
* [Observation](#observation)

## Introduction: Business Problem Section <a name="introduction"></a>

In this Project, I am focusing on identifying the best places to open new Indian Restaurant in **Kuala Lumpur, Malaysia**.
We will be identifying existing Indian Restaurant in neighbourhoods of Kuala Lumpur and identify new locations where there exists opportunities to open new Indian Restaurant.

Thus report will be helpful to clients who are planning to open a new Indian Restaurant in neighbourhoods of **Kuala Lumpur, Malaysia**.

## Data Collection <a name="data"></a>

The Factors we will be focusing on to complete our Business requirement are as follows:
* Identifying the neighbourhoods of Kuala Lumpur.
* Finding existing Indian Restaurant in the neighbourhoods of Kuala Lumpur.
* Exploring new neighbourhoods, where there is opportunity to open new Indian Restaurant.

The Data sources used to generate above points are as below:
* Using **Wikipedia Page** to get data about the neighbourhoods.
* Obtain Coordinates of neighbourhoods.
* Using **Foursquare API** to get the venues data of the neighbourhood.
* Find the best place to open the Indian Restaurant by exploring the clusters.


### Import and install libraries:

In [254]:
!pip install geopy
!pip install geocoder
!pip install folium

distributed 1.21.8 requires msgpack, which is not installed.
You are using pip version 10.0.1, however version 19.2.3 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


distributed 1.21.8 requires msgpack, which is not installed.
You are using pip version 10.0.1, however version 19.2.3 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


distributed 1.21.8 requires msgpack, which is not installed.
You are using pip version 10.0.1, however version 19.2.3 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [255]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")


Libraries imported.


###  Get Neighbourhood Data of Kuala Lumpur

In [256]:
# GET request
data = requests.get("https://en.wikipedia.org/wiki/Category:Suburbs_in_Kuala_Lumpur").text
# Using beautifulsoup object parse data from HTML
soup = BeautifulSoup(data, 'html.parser')


In [257]:
neighbourhood_List = []
# Get data into list 
for area in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighbourhood_List.append(area.text)

In [258]:
# Create DataFrame
DataFrame_KF = pd.DataFrame({"Neighbourhood": neighbourhood_List})
DataFrame_KF.head()

,Neighbourhood
0,Alam Damai
1,"Ampang, Kuala Lumpur"
2,Bandar Menjalara
3,Bandar Sri Permaisuri
4,Bandar Tasik Selatan


### Geographical coordinates of Neighbourhood:

In [259]:
# define a function to get coordinates
def getLatLng(neigh):
    # initialize your variable to None
    cordLatLng = None
    # loop until you get the coordinates
    while(cordLatLng is None):
        g = geocoder.arcgis('{}, Kuala Lumpur, Malaysia'.format(neigh))
        cordLatLng = g.latlng
    return cordLatLng

In [260]:
# call the function to get the coordinates, store in a new list using list comprehension
cordinates = [ getLatLng(neighbourhood) for neighbourhood in DataFrame_KF["Neighbourhood"].tolist() ]

In [261]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(cordinates, columns=['Latitude', 'Longitude'])

In [262]:
# merge the coordinates into the original dataframe
DataFrame_KF['Latitude'] = df_coords['Latitude']
DataFrame_KF['Longitude'] = df_coords['Longitude']

In [263]:
# check the neighborhoods and the coordinates
print(DataFrame_KF.shape)
DataFrame_KF.head()

(70, 3)


,Neighbourhood,Latitude,Longitude
0,Alam Damai,3.057690,101.743880
1,"Ampang, Kuala Lumpur",3.153153,101.700413
2,Bandar Menjalara,3.190350,101.625450
3,Bandar Sri Permaisuri,3.103910,101.712260
4,Bandar Tasik Selatan,3.072750,101.714610


In [264]:
# Save the DataFrame as CSV file
DataFrame_KF.to_csv("DataFrame_KF.csv", index=False)

### 4. Create a map of Kuala Lumpur with neighborhoods superimposed on top

In [265]:
# get the coordinates of Kuala Lumpur
address = 'Kuala Lumpur, Malaysia'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Kuala Lumpur, Malaysiae {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Kuala Lumpur, Malaysiae 3.1516636, 101.6943028.


In [266]:
# create map of Toronto using latitude and longitude values
map_kl = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighbourhood in zip(DataFrame_KF['Latitude'], DataFrame_KF['Longitude'], DataFrame_KF['Neighbourhood']):
    label = '{}'.format(neighbourhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_kl)  
    
map_kl

In [267]:
# save the map as HTML file
map_kl.save('map_kl.html')

### Foursquare API to explore the neighbourhoods.

In [268]:
# define Foursquare Credentials and Version
CLIENT_ID = '4YUI5WA4FLEFU01SXP0IWB3QPMQSRUNH4KHW2LC3XVFAIJHS' # your Foursquare ID
CLIENT_SECRET = '33RA54YANSMMY0YU0RKC41DR41TLSMUSJCRTUSXCDKDU1EPP' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Credentials:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Credentials:
CLIENT_ID: 4YUI5WA4FLEFU01SXP0IWB3QPMQSRUNH4KHW2LC3XVFAIJHS
CLIENT_SECRET:33RA54YANSMMY0YU0RKC41DR41TLSMUSJCRTUSXCDKDU1EPP


In [269]:
radius = 2000
LIMIT = 150

venues = []

for lat, long, neighbourhood in zip(DataFrame_KF['Latitude'], DataFrame_KF['Longitude'], DataFrame_KF['Neighbourhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
     # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighbourhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [270]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighbourhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(6989, 7)


,Neighbourhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Alam Damai,3.05769,101.74388,Pengedar Shaklee Kuala Lumpur,3.061235,101.740696,Supplement Shop
1,Alam Damai,3.05769,101.74388,Machi Noodle 妈子面,3.057695,101.746635,Noodle House
2,Alam Damai,3.05769,101.74388,Minang Tomyam,3.057185,101.749812,Seafood Restaurant
3,Alam Damai,3.05769,101.74388,628火焰鑫茶室,3.058442,101.747947,Chinese Restaurant
4,Alam Damai,3.05769,101.74388,Restoran Ikbal,3.061134,101.750220,Restaurant


In [271]:
venues_df.groupby(["Neighbourhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighbourhood,,,,,,
Alam Damai,100,100,100,100,100,100
"Ampang, Kuala Lumpur",100,100,100,100,100,100
Bandar Menjalara,100,100,100,100,100,100
Bandar Sri Permaisuri,100,100,100,100,100,100
Bandar Tasik Selatan,100,100,100,100,100,100
Bandar Tun Razak,100,100,100,100,100,100
Bangsar,100,100,100,100,100,100
Bangsar Park,100,100,100,100,100,100
Bangsar South,100,100,100,100,100,100


### Find the unique categories from the returned venues

In [272]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 307 uniques categories.


In [273]:
# print out the list of categories
venues_df['VenueCategory'].unique()

array(['Supplement Shop', 'Noodle House', 'Seafood Restaurant',
       'Chinese Restaurant', 'Restaurant',
       'Vegetarian / Vegan Restaurant', 'Breakfast Spot', 'Food Court',
       'Asian Restaurant', 'Park', 'Other Great Outdoors',
       'Dim Sum Restaurant', 'Indian Restaurant', 'Snack Place', 'Spa',
       'Food Truck', 'Bubble Tea Shop', 'Convenience Store',
       'Chinese Breakfast Place', 'Japanese Restaurant', 'Pet Store',
       'Outlet Store', 'Dessert Shop', 'Café', 'Farmers Market',
       'Malay Restaurant', 'Cantonese Restaurant', 'Gym / Fitness Center',
       'Fast Food Restaurant', 'Steakhouse', 'Badminton Court', 'Bakery',
       'Hakka Restaurant', 'Athletics & Sports',
       'Middle Eastern Restaurant', 'Mamak Restaurant', 'Winery',
       'Burger Joint', 'College Bookstore', 'Grocery Store',
       'Halal Restaurant', 'Diner', 'Mexican Restaurant',
       'Monument / Landmark', 'Hostel', 'Latin American Restaurant',
       'Hotel', 'Hotel Pool', 'South India

In [274]:
# check if the results contain "Shopping Mall"
"Indian Restaurant" in venues_df['VenueCategory'].unique()

True

## Analysis <a name="analysis"></a>

### Analyse the Neighbourhoods.

In [275]:
# one hot encoding
kl_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
kl_onehot['Neighbourhoods'] = venues_df['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [kl_onehot.columns[-1]] + list(kl_onehot.columns[:-1])
kl_onehot = kl_onehot[fixed_columns]

print(kl_onehot.shape)
kl_onehot.head()

(6989, 308)


,Neighbourhoods,Accessories Store,Adult Boutique,African Restaurant,American Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Badminton Court,Bagel Shop,Bakery,Bar,Baseball Field,Basketball Court,Bed & Breakfast,Beer Bar,Beer Garden,Betting Shop,Bistro,Board Shop,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Building,Burger Joint,Business Service,Café,Camera Store,Campground,Candy Store,Cantonese Restaurant,Casino,Chettinad Restaurant,Chinese Breakfast Place,Chinese Restaurant,Circus,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Bookstore,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dive Shop,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Event Space,Exhibit,Fabric Shop,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Fishing Store,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,Frame Store,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Government Building,Grilled Meat Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hainan Restaurant,Hakka Restaurant,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,High School,Hill,Historic Site,History Museum,Hockey Arena,Hong Kong Restaurant,Hookah Bar,Hostel,Hot Spring,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Housing Development,Hunan Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Indoor Play Area,Irish Pub,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kushikatsu Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Lounge,Malay Restaurant,Mamak Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Mosque,Motel,Motorcycle Shop,Mountain,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,Night Market,Nightclub,Noodle House,North Indian Restaurant,Office,Optical Shop,Other Great Outdoors,Outdoor Event Space,Outlet Store,Pakistani Restaurant,Palace,Park,Performing Arts Venue,Pet Café,Pet Store,Pharmacy,Piano Bar,Pie Shop,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Print Shop,Pub,Public Art,Racetrack,Ramen Restaurant,Recording Studio,Recreation Center,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Rock Climbing Spot,Rock Club,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Satay Restaurant,Scandinavian Restaurant,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Ski Chalet,Ski Lodge,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,South Indian Restaurant,Spa,Spanish Restaurant,Speakeasy,Spiritual Center,Sporting Goods Shop,Sports Bar,Sports Club,Sri Lankan Restaurant,Stadium,Stationery Store,Steakhouse,Street Food Gathering,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Szechuan Restaurant,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Temple,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Restaurant,Thrift / Vinta

### Group rows by neighborhood and take the mean of the frequency of occurrence of each category.

In [276]:
kl_grouped = kl_onehot.groupby(["Neighbourhoods"]).mean().reset_index()

print(kl_grouped.shape)
kl_grouped

(70, 308)


,Neighbourhoods,Accessories Store,Adult Boutique,African Restaurant,American Restaurant,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Badminton Court,Bagel Shop,Bakery,Bar,Baseball Field,Basketball Court,Bed & Breakfast,Beer Bar,Beer Garden,Betting Shop,Bistro,Board Shop,Bookstore,Botanical Garden,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Buffet,Building,Burger Joint,Business Service,Café,Camera Store,Campground,Candy Store,Cantonese Restaurant,Casino,Chettinad Restaurant,Chinese Breakfast Place,Chinese Restaurant,Circus,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Bookstore,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dive Shop,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Event Space,Exhibit,Fabric Shop,Farm,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Fishing Store,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,Fountain,Frame Store,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Government Building,Grilled Meat Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hainan Restaurant,Hakka Restaurant,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,High School,Hill,Historic Site,History Museum,Hockey Arena,Hong Kong Restaurant,Hookah Bar,Hostel,Hot Spring,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Housing Development,Hunan Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Indoor Play Area,Irish Pub,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Kushikatsu Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Lounge,Malay Restaurant,Mamak Restaurant,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Mosque,Motel,Motorcycle Shop,Mountain,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,Night Market,Nightclub,Noodle House,North Indian Restaurant,Office,Optical Shop,Other Great Outdoors,Outdoor Event Space,Outlet Store,Pakistani Restaurant,Palace,Park,Performing Arts Venue,Pet Café,Pet Store,Pharmacy,Piano Bar,Pie Shop,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Print Shop,Pub,Public Art,Racetrack,Ramen Restaurant,Recording Studio,Recreation Center,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Rock Climbing Spot,Rock Club,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Satay Restaurant,Scandinavian Restaurant,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Ski Chalet,Ski Lodge,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,South Indian Restaurant,Spa,Spanish Restaurant,Speakeasy,Spiritual Center,Sporting Goods Shop,Sports Bar,Sports Club,Sri Lankan Restaurant,Stadium,Stationery Store,Steakhouse,Street Food Gathering,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Szechuan Restaurant,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Temple,Tennis Court,Thai Restaurant,Theater,Theme Park,Theme Restaurant,Thrift / Vinta

In [277]:
len(kl_grouped[kl_grouped["Indian Restaurant"] > 0])

55

### Create a new DataFrame for Indian Restaurant data only

In [278]:
kl_rest = kl_grouped[["Neighbourhoods","Indian Restaurant"]]

In [279]:
kl_rest.head()

,Neighbourhoods,Indian Restaurant
0,Alam Damai,0.03
1,"Ampang, Kuala Lumpur",0.00
2,Bandar Menjalara,0.01
3,Bandar Sri Permaisuri,0.03
4,Bandar Tasik Selatan,0.02


### 7. Cluster Neighborhoods
Run k-means to cluster the neighborhoods in Kuala Lumpur into 3 clusters.

In [280]:
# set number of clusters
kclusters = 3

kl_clustering = kl_rest.drop(["Neighbourhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(kl_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 0, 0, 2, 2, 2, 1, 1, 0, 0])

In [281]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
kl_merged = kl_rest.copy()

# add clustering labels
kl_merged["Cluster Labels"] = kmeans.labels_

In [282]:
kl_merged.rename(columns={"Neighbourhoods": "Neighbourhood"}, inplace=True)
kl_merged.head()

,Neighbourhood,Indian Restaurant,Cluster Labels
0,Alam Damai,0.03,2
1,"Ampang, Kuala Lumpur",0.00,0
2,Bandar Menjalara,0.01,0
3,Bandar Sri Permaisuri,0.03,2
4,Bandar Tasik Selatan,0.02,2


In [283]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
kl_merged = kl_merged.join(DataFrame_KF.set_index("Neighbourhood"), on="Neighbourhood")

print(kl_merged.shape)
kl_merged.head() # check the last columns!

(70, 5)


,Neighbourhood,Indian Restaurant,Cluster Labels,Latitude,Longitude
0,Alam Damai,0.03,2,3.057690,101.743880
1,"Ampang, Kuala Lumpur",0.00,0,3.153153,101.700413
2,Bandar Menjalara,0.01,0,3.190350,101.625450
3,Bandar Sri Permaisuri,0.03,2,3.103910,101.712260
4,Bandar Tasik Selatan,0.02,2,3.072750,101.714610


In [284]:
# sort the results by Cluster Labels
print(kl_merged.shape)
kl_merged.sort_values(["Cluster Labels"], inplace=True)
kl_merged

(70, 5)


,Neighbourhood,Indian Restaurant,Cluster Labels,Latitude,Longitude
34,Kuchai Lama,0.010000,0,3.090740,101.677330
29,Jinjang,0.000000,0,3.209500,101.658740
31,"Kampung Baru, Kuala Lumpur",0.000000,0,3.165460,101.710280
32,Kampung Datuk Keramat,0.010000,0,3.166400,101.730460
33,Kepong,0.000000,0,3.217500,101.637630
37,Medan Tuanku,0.010000,0,3.159260,101.698340
39,Mont Kiara,0.010000,0,3.165320,101.652430
41,"Pudu, Kuala Lumpur",0.010000,0,3.133540,101.713070
42,Putrajaya,0.000000,0,3.125862,101.718624
43,Salak South,0.010000,0,3.081020,101.697240


### Finally, let's visualize the resulting clusters

In [285]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(kl_merged['Latitude'], kl_merged['Longitude'], kl_merged['Neighbourhood'], kl_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [286]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

### Examine Clusters
#### Cluster 0

In [287]:
kl_merged.loc[kl_merged['Cluster Labels'] == 0]

,Neighbourhood,Indian Restaurant,Cluster Labels,Latitude,Longitude
34,Kuchai Lama,0.01,0,3.090740,101.677330
29,Jinjang,0.00,0,3.209500,101.658740
31,"Kampung Baru, Kuala Lumpur",0.00,0,3.165460,101.710280
32,Kampung Datuk Keramat,0.01,0,3.166400,101.730460
33,Kepong,0.00,0,3.217500,101.637630
37,Medan Tuanku,0.01,0,3.159260,101.698340
39,Mont Kiara,0.01,0,3.165320,101.652430
41,"Pudu, Kuala Lumpur",0.01,0,3.133540,101.713070
42,Putrajaya,0.00,0,3.125862,101.718624
43,Salak South,0.01,0,3.081020,101.697240


#### Cluster 1

In [288]:
kl_merged.loc[kl_merged['Cluster Labels'] == 1]

,Neighbourhood,Indian Restaurant,Cluster Labels,Latitude,Longitude
25,"Federal Hill, Kuala Lumpur",0.06,1,3.136370,101.685640
46,Sentul Raya,0.07,1,3.187431,101.691453
7,Bangsar Park,0.07,1,3.134780,101.672620
6,Bangsar,0.08,1,3.129200,101.678440
40,Pantai Dalam,0.09,1,3.094760,101.667470
35,Lembah Pantai,0.05,1,3.121202,101.663899
68,Titiwangsa,0.05,1,3.180670,101.703220
21,Damansara Town Centre,0.06,1,3.138759,101.684046
22,"Damansara, Kuala Lumpur",0.06,1,3.138759,101.684046
11,Brickfields,0.06,1,3.129160,101.684060


#### Cluster 2

In [289]:
kl_merged.loc[kl_merged['Cluster Labels'] == 2]

,Neighbourhood,Indian Restaurant,Cluster Labels,Latitude,Longitude
5,Bandar Tun Razak,0.020000,2,3.082800,101.722810
62,Taman P. Ramlee,0.021505,2,3.193600,101.705980
63,Taman Sri Sinar,0.020833,2,3.190070,101.652930
4,Bandar Tasik Selatan,0.020000,2,3.072750,101.714610
58,Taman Len Seng,0.030000,2,3.069080,101.742870
3,Bandar Sri Permaisuri,0.030000,2,3.103910,101.712260
10,"Batu, Kuala Lumpur",0.020000,2,3.135760,101.708370
54,Taman Cheras Hartamas,0.030000,2,3.082630,101.746710
13,Bukit Jalil,0.020000,2,3.057800,101.689650
51,Sri Petaling,0.020000,2,3.072600,101.682520


### Observations: <a name="observation"></a>

From our analysis, we can conclude that most of the **Indian Restaurants** are present in Cluster 1. Whereas Cluster 2 is moderately concentrated with **Indian Restaurant**. Cluster 0 has the least **Indian Restaurants**
As there are number of Indian Restaurants already in existense in cluster 1 and 2, it provides an intense competition to open new restaurant in that neighbourhood and hence will be a bad idea to open a new Restaurant over there.

New clients who are planning to open an **Indian Restaurant** can look for opportunities in cluster 0 and can also look for some options in cluster 2, as there is almost no competition, to very less, it would provide a great opportunity for business.Thus we conclude our analysis. 